In [0]:
# RNN poem generatior

In [0]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


In [0]:
# one letter at a time, we run multiple times and the play is generated.

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file

'/root/.keras/datasets/shakespeare.txt'

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(len(text))

1115394


In [0]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# preprocessing , encoding to integers.

vocab = sorted(set(text))
len(vocab)

65

In [0]:
# create a dict of the vocab and their representetive number.
char2idx = {u:i for i, u in enumerate(vocab)}
len(char2idx)
idx2char = np.array(vocab)
idx2char

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [0]:
arr = text_to_int('Speak, speak.')

In [0]:
def int_to_text(arr):
  try:
    arr = arr.numpy()
  except:
    pass
  return ''.join(idx2char[arr])

In [0]:
txt = int_to_text(arr)
txt

'Speak, speak.'

In [0]:
# shift words by one character to create training samples

seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
examples_per_epoch

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
# lengths of 101 are picked

In [0]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1] # HELL
  target_text = chunk[1:] # ELLO
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [0]:
for x, y in dataset.take(2):
  print(int_to_text(x))
  print(int_to_text(y))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024


BUFFER_SIZE = 1000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequences=True,
                                                    stateful=True,
                                                    recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size) # probability for the next character
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 65)


In [0]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-0.00082702  0.00200509 -0.00070652 ...  0.00045198 -0.00115732
    0.00677575]
  [-0.00279657  0.00202235  0.00024598 ... -0.00221867  0.0017723
    0.00596637]
  [-0.00454159  0.00581424 -0.00274956 ...  0.00158515  0.00209609
    0.01188151]
  ...
  [-0.01290994  0.01519141 -0.00305946 ...  0.00227756 -0.00248681
    0.00629189]
  [-0.01630002  0.01841058 -0.00321782 ...  0.0003033  -0.00040643
    0.00392205]
  [-0.01577934  0.01321387 -0.00107076 ...  0.00043978  0.00105818
    0.00192802]]

 [[-0.00169298 -0.00118001 -0.00030001 ... -0.0038305   0.00085708
    0.00166719]
  [ 0.00277293  0.00186329  0.00014444 ...  0.00173474 -0.00104293
    0.00529115]
  [ 0.00455769 -0.00424499 -0.00598387 ...  0.00434343  0.00631629
    0.00421247]
  ...
  [-0.00432417  0.01549965 -0.01277741 ...  0.00461921 -0.00503589
    0.00742809]
  [-0.00141547  0.01122486 -0.00647786 ... -0.00149425 -0.00345704
    0.00674018]
  [-0.001207    0.01106475 -0.00108161 ... -0.0031883  -0.00

In [0]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# for every character there is a prediction

100
tf.Tensor(
[[-0.00082702  0.00200509 -0.00070652 ...  0.00045198 -0.00115732
   0.00677575]
 [-0.00279657  0.00202235  0.00024598 ... -0.00221867  0.0017723
   0.00596637]
 [-0.00454159  0.00581424 -0.00274956 ...  0.00158515  0.00209609
   0.01188151]
 ...
 [-0.01290994  0.01519141 -0.00305946 ...  0.00227756 -0.00248681
   0.00629189]
 [-0.01630002  0.01841058 -0.00321782 ...  0.0003033  -0.00040643
   0.00392205]
 [-0.01577934  0.01321387 -0.00107076 ...  0.00043978  0.00105818
   0.00192802]], shape=(100, 65), dtype=float32)


In [0]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-0.00082702  0.00200509 -0.00070652 -0.00134229  0.00200452  0.00123997
 -0.00380347  0.0005456   0.00391901 -0.00252769 -0.00254544 -0.0037947
 -0.0027279   0.00414711  0.0009107  -0.00168401  0.00090094 -0.00163784
  0.00205354  0.00309464 -0.00372314  0.00067793  0.0016525  -0.00930487
  0.00256955 -0.00215466 -0.00245249  0.00246958 -0.00426134  0.00099242
 -0.00142198 -0.00233984  0.00012841 -0.00495532 -0.0042601   0.00110647
 -0.00562175 -0.00417898 -0.00545525  0.00144647 -0.003159   -0.00379408
 -0.00347164  0.00324371 -0.00047511  0.00023025 -0.0011632  -0.00147226
 -0.00417836 -0.00218375  0.00068305 -0.00155582  0.0066727  -0.00146308
 -0.0044701  -0.00496119 -0.00097079 -0.00047026 -0.00419251  0.00628326
 -0.00132167 -0.0005822   0.00045198 -0.00115732  0.00677575], shape=(65,), dtype=float32)


In [0]:
sampled_indices = tf.random.categorical(pred, num_samples=1)
print(sampled_indices)
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
print(sampled_indices)
predicted_chars = int_to_text(sampled_indices)

print(predicted_chars)
print(len(predicted_chars))

tf.Tensor(
[[40]
 [64]
 [63]
 [64]
 [63]
 [ 5]
 [26]
 [62]
 [41]
 [59]
 [ 6]
 [ 5]
 [17]
 [12]
 [42]
 [37]
 [ 5]
 [20]
 [ 7]
 [44]
 [ 2]
 [26]
 [54]
 [25]
 [55]
 [35]
 [64]
 [20]
 [30]
 [50]
 [58]
 [ 3]
 [17]
 [23]
 [20]
 [38]
 [26]
 [59]
 [20]
 [62]
 [45]
 [17]
 [38]
 [41]
 [63]
 [33]
 [47]
 [31]
 [22]
 [16]
 [ 6]
 [61]
 [40]
 [19]
 [44]
 [ 8]
 [33]
 [13]
 [ 1]
 [19]
 [12]
 [52]
 [18]
 [42]
 [45]
 [45]
 [32]
 [30]
 [10]
 [63]
 [18]
 [27]
 [38]
 [52]
 [ 4]
 [ 7]
 [56]
 [22]
 [10]
 [10]
 [47]
 [48]
 [27]
 [26]
 [13]
 [48]
 [14]
 [59]
 [33]
 [52]
 [14]
 [22]
 [28]
 [11]
 [62]
 [32]
 [32]
 [48]
 [21]
 [52]], shape=(100, 1), dtype=int64)
[40 64 63 64 63  5 26 62 41 59  6  5 17 12 42 37  5 20  7 44  2 26 54 25
 55 35 64 20 30 50 58  3 17 23 20 38 26 59 20 62 45 17 38 41 63 33 47 31
 22 16  6 61 40 19 44  8 33 13  1 19 12 52 18 42 45 45 32 30 10 63 18 27
 38 52  4  7 56 22 10 10 47 48 27 26 13 48 14 59 33 52 14 22 28 11 62 32
 32 48 21 52]
bzyzy'Nxcu,'E?dY'H-f!NpMqWzHRlt$EKHZNuHxgEZcyUiSJD,w

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 15s 87ms/step - loss: 2.6132
Epoch 2/40
172/172 [==============================] - 15s 86ms/step - loss: 1.9304
Epoch 3/40
172/172 [==============================] - 15s 87ms/step - loss: 1.6832
Epoch 4/40
172/172 [==============================] - 15s 87ms/step - loss: 1.5384
Epoch 5/40
172/172 [==============================] - 15s 88ms/step - loss: 1.4473
Epoch 6/40
172/172 [==============================] - 15s 88ms/step - loss: 1.3859
Epoch 7/40
172/172 [==============================] - 15s 88ms/step - loss: 1.3384
Epoch 8/40
172/172 [==============================] - 15s 89ms/step - loss: 1.2981
Epoch 9/40
172/172 [==============================] - 15s 89ms/step - loss: 1.2629
Epoch 10/40
172/172 [==============================] - 15s 89ms/step - loss: 1.2282
Epoch 11/40
172/172 [==============================] - 15s 90ms/step - loss: 1.1939
Epoch 12/40
172/172 [==============================] - 15s 90ms/step - loss: 1.1606
E

In [0]:
# rebuild model for predicting

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)


In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
# or load any crappy checkpoint you wnt
'''
num = 40
model.load_weights(tf.train.latest_checkpoint('./training_checkpoints/cpkt_' + str(num)))
model.build(tf.TensorShape([1, None]))
'''

In [0]:
def generate_text(model, start_string):
  num_generate = 800 # predict 800 words

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0 # low more predictable, high more suprising. Must be optimized

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return(start_string + ''.join(text_generated))

In [0]:
inp = input('Starting String...')
print(generate_text(model, inp))

Starting String...oh n
oh no place herein all this while hath made before,
For put it yet, to you, courage, Kate, I will.
Father, you'll know more dead or two:
I have kill'd King Henry's friendly voice.
Yer, first Duke of King Lewis worship,
And thy desires the world with the world
Marry'd to appear a fortnight shall be my constable.

GREMIO:
I bray me to my best know her good unnatural
countryman.

PETRUCHIO:
Believe me, thou'rt to crave dwell on my part in home-business,
Lest they have put me in your clothes; and here be the faults from some form.

CLAUDIO:
I have to thee,
For 'tis as ever ready to come again.

Page:
Your Son, the glorious sun to one:
This must be so received. Far I not right;
Nor need you, gentlemen, adieu.

GONZALO:

ARILA:
Down with the swords and in love is not at the way;
Not thus; for Warwic
